In [2]:
import matplotlib.pyplot as plt
import matplotlib as mtp
import math
import time
import numpy as np
from numpy import *
from collections import Counter
from scipy import interpolate
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
import sys
import os
import pylab
from matplotlib import rc
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
#import illustris_python as il
import seaborn as sns
import pandas as pan
import astropy
from pyagn import sed
from astropy import units as u

In [3]:
L_box=50.
h=0.6774
cte_G=6.67408*10**-8 ## cm3 g-1 s-2                                                                                                                                               
cte_m_p=1.6726*10**-24 ## g                                                                                                                                                       
cte_eps_r=0.1
cte_sigma_t=6.6524*10**-25 ## cm2                                                                                                                                                 
cte_c=2.998*10**10 ## cm s-1 

In [4]:
## Parameters for the SED class
"""
| parameter | type    | description                           |default         |
| --------- | ------- | ---------------------------------------|------------- |
| `M`     | float  | Black Hole mass in solar masses.       | 1e8
| `mdot`  | float | Black Hole accretion rate in Eddington units. | 0.5       |
| `astar` | float  | Black Hole dimensionless spin absolute value.                | 0   |
| `astar_sign`  | int | +1 for prograde rotation, -1 for retrograde. | +1 |
| `reprocessing`  | boolean | True to include reprocessing, False otherwise. | True |
| `hard_xray_fraction`  | float | Dissipated corona luminosity in Eddington units.  | 0.02 |
| `corona_electron_energy`  | float | Electron temperature for the hot Comptonisation component in keV | 100 |
| `warm_electron_energy`  | float | Electron temperature for the warm Comptonisation component in keV.  | 0.2 |
| `warm_photon_index`  | float | The spectral index $\Gamma$ of the warm Comptonisation component. | 2.5 |
| `reflection_albedo`  | float | reflection albedo for the reprocessed flux | 0.3 |
"""

'\n| parameter | type    | description                           |default         |\n| --------- | ------- | ---------------------------------------|------------- |\n| `M`     | float  | Black Hole mass in solar masses.       | 1e8\n| `mdot`  | float | Black Hole accretion rate in Eddington units. | 0.5       |\n| `astar` | float  | Black Hole dimensionless spin absolute value.                | 0   |\n| `astar_sign`  | int | +1 for prograde rotation, -1 for retrograde. | +1 |\n| `reprocessing`  | boolean | True to include reprocessing, False otherwise. | True |\n| `hard_xray_fraction`  | float | Dissipated corona luminosity in Eddington units.  | 0.02 |\n| `corona_electron_energy`  | float | Electron temperature for the hot Comptonisation component in keV | 100 |\n| `warm_electron_energy`  | float | Electron temperature for the warm Comptonisation component in keV.  | 0.2 |\n| `warm_photon_index`  | float | The spectral index $\\Gamma$ of the warm Comptonisation component. | 2.5 |\n| `

In [5]:
def plot_total_flux(self, distance, color, ax = None):
        """                                                                                                                                                                              
        Plot total flux in energy units.                                                                                                                                                 
        yaxis: EL_E [ keV keV / s / keV]                                                                                                                                                 
        xaxis: E [keV]                                                                                                                                                                   
        """
        if(ax is None):
            print("creating figure.")
            fig, ax = plt.subplots()
        
        #flux = self.corona_flux(distance) + self.disk_flux(distance) + self.warm_flux(distance)
        flux = self.total_flux(distance)
        return ax,list(self.energy_range),list(flux)

In [6]:
#### the setups
M=[1e4,1e5,1e6,1e7,1e8] ## BH mass
mdot=[0.25,0.5,0.75,1.] ## BH accretion rate in Edd units
astar=0. ## BH dimensionless spin absolute value.
hard_xray_fraction=0.02
corona_electron_energy=100
warm_electron_energy=0.2
warm_photon_index=2.5
reflection_albedo=0.3


#### the figure 
col0=sns.color_palette("mako")[0]
col1=sns.color_palette("mako")[1]
col2=sns.color_palette("mako")[2]
col3=sns.color_palette("mako")[3]
col4=sns.color_palette("mako")[4]
col5=sns.color_palette("mako")[5]
colorr=[col0,col1,col2,col3,col4,col5]
mtp.rcParams['figure.figsize'] = [11.5, 9.5]

## first units
gs1=gridspec.GridSpec(2,1)
gs1.update(left=0.1, right=0.4,bottom=0.3,top=0.8, hspace=0.0001,wspace=0.0001)

mtp.rcParams['figure.figsize'] = [14, 14]
plt.figure()


distance = 10.0
for k in range(2):
    print(k)
    if k==0: ## varying BH mass in keV units
        ax=plt.subplot(gs1[0])
        #ax.tick_params(axis='both',which='both',direction="in",bottom=True,top=True,left=True,right=True)
        for kk in range(5):
            my_sed=sed.SED(M=M[kk],mdot=mdot[1],astar=astar,hard_xray_fraction=hard_xray_fraction,corona_electron_energy=corona_electron_energy,warm_electron_energy=warm_electron_energy,reflection_albedo=reflection_albedo,reprocessing=True)


            data=plot_total_flux(my_sed,distance, color=colorr[kk],ax = ax)
            energy_kev=data[1]
            flux=np.array(data[2])
            ## To plot in keV units
            ## plt.plot(energy_kev,flux) 
            ## plt.ylabel(r'$\rm E F_{E}\,\,(Photons / cm^{2} / s / keV) $')
            ## plt.xlabel(r'E (keV)')
             ## To plot in nuFnu as a function of lambda.
            x_lambda=[1.24*1e-3/i for i in energy_kev]              # convert from kev to microns
            
            # convert microns to hz or sth related
            #convert flux

            #convert to luminosity
            #flux = flux*(4*np.pi*(distance**2))

            #units of solar luminosity
            #flux = flux/(3.826e+33)

            # okay so see table (https://hea-www.harvard.edu/~pgreen/figs/Conversions.pdf)
            #this converts from f_E to F_nu, but not sure about the 10^17, so gonna change to energy_kev like in table, 
            #seems like converting the wavelength to nu here, but this should be in kev as far as I am aware. 
            # actually this should already be included in the output. The output is E*F_E
            
            #y_F=[6.63*1e-27*flux[i]*2.42*1e17*x_lambda[i] for i in range(len(flux))] 

            #y_F=6.63*1e-27*flux
            #now flux is in f-nu

            #now convert f_nu to f_lambda, by converting lambda to armstrong
            #y_F=[y_F[i]*(3e+18)/(x_lambda[i]*10000)**2 for i in range(len(flux))] 


            #or convert directly to f_lamba 
            y_F=[1.29*1e-10*flux[i]*(energy_kev[i]**2) for i in range(len(flux))] 

            #now convert to luminosity
            y_F = np.array(y_F)*(4*np.pi*(distance**2))

            y_F = np.array(y_F)
            #now convert to solar luminosities
            y_F = y_F/(3.826e+33)


            #plt.plot(x_lambda,y_F)
    plt.loglog()
    ax.set_xticklabels([])
    #plt.xlim(1e-4,1e2)
    #plt.ylim(1e-18,1e-8)
    #plt.ylabel(r'$\rm \nu\, F_{\nu}\,\,(erg / cm^{2} / s) $')
    #plt.ylabel(r'$\rm \nu\, F_{\nu}\,\,(erg  / s) $')
    plt.ylabel(r'$\rm  F_{\nu}\,\,(L_{\odot}) $')
    plt.xlabel(r'$\rm \lambda \,\,(\mu m)$')



    if k==1: ## varying BH mass in keV units
        ax=plt.subplot(gs1[1])
        ax.tick_params(axis='both',which='both',direction="in",bottom=True,top=True,left=True,right=True)
        for kk in range(4):
            my_sed=sed.SED(M=M[1],mdot=mdot[kk],astar=astar,hard_xray_fraction=hard_xray_fraction,corona_electron_energy=corona_electron_energy,warm_electron_energy=warm_electron_energy,reflection_albedo=reflection_albedo,reprocessing=True)
            
            data=plot_total_flux(my_sed,distance,color=colorr[kk], ax = ax)
            energy_kev=data[1]
            flux=np.array(data[2])
            ## To plot in keV units
            ## plt.plot(energy_kev,flux) 
            ## plt.ylabel(r'$\rm E F_{E}\,\,(Photons / cm^{2} / s / keV) $')
            ## plt.xlabel(r'E (keV)')
            
            ## To plot in nuFnu as a function of lambda.
            x_lambda=[1.24*1e-3/i for i in energy_kev]              # convert from kev to microns
            
            # convert microns to hz or sth related
            #convert flux

            #convert to luminosity
            #flux = flux*(4*np.pi*(distance**2))

            #units of solar luminosity
            #flux = flux/(3.826e+33)

            # okay so see table (https://hea-www.harvard.edu/~pgreen/figs/Conversions.pdf)
            #this converts from f_E to F_nu, but not sure about the 10´^17, so gonna change to energy_kev like in table, 
            #seems like converting the wavelength to nu here, but this should be in kev as far as I am aware. 
            # actually this should already be included in the output. The output is E*F_E
            
            #y_F=[6.63*1e-27*flux[i]*2.42*1e17*x_lambda[i] for i in range(len(flux))] 

            #y_F=6.63*1e-27*flux
            #now flux is in f-nu

            #now convert f_nu to f_lambda, by converting lambda to armstrong
            #y_F=[y_F[i]*(3e+18)/(x_lambda[i]*10000)**2 for i in range(len(flux))] 


            #or convert directly to f_lamba 
            y_F=[1.29*1e-10*flux[i]*(energy_kev[i]**2) for i in range(len(flux))] 

            #now convert to luminosity
            y_F = np.array(y_F)*(4*np.pi*(distance**2))

            y_F = np.array(y_F)
            #now convert to solar luminosities
            y_F = y_F/(3.826e+33)



            #plt.plot(x_lambda,y_F)
            
            
            
    plt.loglog()
    #plt.xlim(1e-4,1e2)
    #plt.ylim(1e-18,1e-8)
    #plt.ylabel(r'$\rm \nu\, F_{\nu}\,\,(erg / cm^{2} / s) $')
    #plt.ylabel(r'$\rm \nu\, F_{\nu}\,\,(erg  / s) $')
    plt.ylabel(r'$\rm  F_{\nu}\,\,(L_{\odot}) $')
    plt.xlabel(r'$\rm \lambda \,\,(\mu m)$')
    
    

plt.savefig('Example_AGN_sed.pdf')


: 

: 